# Quickstart: Generating Insights using Langchain
Here is how you can setup your own insight generator

Make sure you have your `SERPER_API_KEY` and `OPEN_AI_API_KEY` in your env

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.agents.tools import Tool
import os

llm = ChatOpenAI(temperature=0, openai_api_key=os.environ.get("OPEN_AI_API_KEY"), model="gpt-4-0613")

## Tools for our agent
We had decided to give our agents the ability to
- Search for a query using the web
- Scrape a page to find out more info

In [2]:
# Scraping tool
from bs4 import BeautifulSoup
import requests

banned_sites = ["calendar.google.com", "researchgate.net"]

def scrape_page(url: str):
    if any(substring in url for substring in banned_sites):
        print("Skipping site: {}".format(url))
        return None
    
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive',
        }
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        text = " ".join([t.get_text() for t in soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])])
        return text.replace('|','')
    except requests.RequestException as e:
        print(f"Failed to fetch {url}. Error: {e}")
        return None

In [3]:
# SBert Summarizer
from summarizer.sbert import SBertSummarizer
summarizer = SBertSummarizer('paraphrase-MiniLM-L6-v2')

def summarize(text, num_sentences=3):
    return summarizer(text, num_sentences=num_sentences)

In [4]:
# Search tool
from typing import Any, List, Literal
import requests
import os

k: int = 5
gl: str = "us"
hl: str = "en"
tbs = None
num_sentences = 7
serper_api_key=os.environ.get("SERPER_API_KEY")
search_type: Literal["news", "search", "places", "images"] = "search"

def serper_search(
        search_term: str, search_type: str = "search", **kwargs: Any
    ) -> dict:
    headers = {
        "X-API-KEY": serper_api_key or "",
        "Content-Type": "application/json",
    }
    params = {
        "q": search_term,
        **{key: value for key, value in kwargs.items() if value is not None},
    }
    response = requests.post(
        f"https://google.serper.dev/{search_type}", headers=headers, params=params
    )
    response.raise_for_status()
    search_results = response.json()
    return search_results

def parse_snippets(results: dict) -> List[str]:
    result_key_for_type = {
        "news": "news",
        "places": "places",
        "images": "images",
        "search": "organic",
    }
    snippets = []
    if results.get("answerBox"):
        answer_box = results.get("answerBox", {})
        if answer_box.get("answer"):
            snippets.append(answer_box.get("answer"))
        elif answer_box.get("snippet"):
            snippets.append(answer_box.get("snippet").replace("\n", " "))
        elif answer_box.get("snippetHighlighted"):
            snippets.append(answer_box.get("snippetHighlighted"))

    if results.get("knowledgeGraph"):
        kg = results.get("knowledgeGraph", {})
        title = kg.get("title")
        entity_type = kg.get("type")
        if entity_type:
            snippets.append(f"{title}: {entity_type}.")
        description = kg.get("description")
        if description:
            snippets.append(description)
        for attribute, value in kg.get("attributes", {}).items():
            snippets.append(f"{title} {attribute}: {value}.")

    for result in results[result_key_for_type[search_type]][:k]:
        if "snippet" in result:
            page = scrape_page(result["link"])
            if page is None:
                snippets.append(f"Title: {result['title']}\nLink: {result['link']}\nSnippet: {result['snippet']}\n")
            else:
                summarized_page = summarize(page, num_sentences=num_sentences)
                if len(summarized_page) == 0:
                    summarized_page = "None"
                snippets.append(f"Title: {result['title']}\nLink: {result['link']}\nSnippet: {result['snippet']}\nSummarized Page: <p>{summarized_page}</p>")

    if len(snippets) == 0:
        return ["No good Google Search Result was found"]
    return snippets

def parse_results(results: dict) -> str:
        snippets = parse_snippets(results)
        results_string = ""
        for idx, val in enumerate(snippets):
            results_string += f"<result{idx}>\n{val}\n</result{idx}>\n\n"
        return results_string

def custom_search(query: str, parse=True, **kwargs: Any):
    results = serper_search(
            search_term=query,
            gl=gl,
            hl=hl,
            num=k,
            tbs=tbs,
            search_type=search_type,
            **kwargs,
        )
    return parse_results(results)

## Preparing mock text input

In [63]:
# Way to generate a random test input using transcripts from Lex Fridman's podcast
# Make sure you have the transcripts downloaded in the folder lex_whisper_transcripts

import test_on_lex

transcripts = test_on_lex.load_lex_transcripts(random_n=10, transcript_folder="./lex_whisper_transcripts/", chunk_time_seconds=20)

import random
def generate_test_input():
    idx = random.randint(0, 10)
    key = list(transcripts.keys())[idx]
    transcript = transcripts[key]
    trans_idx = random.randint(10, len(transcript)-10)
    latest = transcript[trans_idx:trans_idx+7]
    prev_transcripts, curr_transcripts = str.join(",", list(latest[0:5])), latest[5]
    return prev_transcripts + "\n" + curr_transcripts

generate_test_input()

Processing episode_083_large...
Processing episode_149_large...
Processing episode_060_large...
Processing episode_286_large...
Processing episode_123_large...
Processing episode_062_large...
Processing episode_031_large...
Processing episode_013_large...
Processing episode_026_large...
Processing episode_231_large...


" There's a strong currents of opinion. So contrary to what the position of him as a dictator, he wouldn't last if he was unpopular, number one. Number two, Russia is much more divided than people know. There's other factors in Russia. There are always tensions around the Kremlin, who has power, who doesn't have power., That's been going on for 100 years. But the factions in Russia are very much there. So when people refer to Russia as Putin, they're mistaken. And they do this regularly in the New York papers and all this. They say, Putin did this, Putin did that, Putin's doing that, but it's Russia that's doing it., And that's what, there's a distinction there that I, it's changed. In the old days, I would read about Khrushchev, but it was never Khrushchev personally. It was about the Soviet Union. There was respect for a country. And now when it started to get personal with Putin, it changed and our thinking changed in a negative way., We no longer respected it as a country, we were 

## Initialize master and worker agents

In [ ]:
generate_master_prompt = lambda x: f"""
You are the master agent of "Convoscope". "Convoscope" is a tool that listens to a user's live conversation and enhances their conversation by providing them with real time "Insights". The "Insights" you generate should aim to lead the user to deeper understanding, broader perspectives, new ideas, more accurate information, better replies, and enhanced conversations. 

[Your Objective]
"Convoscope" is a multi-agent system in which you are the master agent. You will be given direct access to a live stream of transcripts from the user's conversation. Your goal is to utilize your knowledge and tools to generate "Insights" for the user.

[Your Tools]
You have access to "Agents", which are like workers in your team that can help you do certain tasks. Imagine you are a human manager and your agents as human workers. You can assign tasks to your agents and they will help you complete the tasks. Speak to them like how you would speak to a human worker, give detailed context and instructions.

<Task start>
It's now time to generate an "Insight" for the following conversation transcript. The "Insight" should provide additional understanding beyond what is currently being said in the transcript, it shouldn't be plainly repeating what is being said in the transcripts. If a tool or agent fails to fulfill your request, don't run the same request on the same agent again. 

In your initial thought, you should first write down a plan to generate the "Insight". The plan should include
1. Read the incoming conversation transcript and identify the best "Insight" you could generate to enhance the user's conversation.  Come up with a general description of the "Insight" to generate.
2. What tool(s), agent(s), information you need to generate the "Insight".
3. A final step to almagamate your and your worker agent's work to generate the "Insight". The insight should be summarized within 12 words and be in the format `Insight: {{Insert your "Insight" here}}`
<Task end>

<Transcript start>{x}<Transcript end>
"""

In [ ]:
from langchain.agents import initialize_agent
from langchain.agents import load_tools
from langchain.tools import StructuredTool
from langchain.agents import AgentType

agents = [[]]

statistician_agent = initialize_agent([
        Tool(
            name="Search_Engine",
            func=custom_search,
            description="Use this tool to search for statistics and facts about a topic. Pass this specific targeted queries and/or keywords to quickly search the WWW to retrieve vast amounts of information on virtually any topic, spanning from academic research and navigation to history, entertainment, and current events.",
        ),
    ], llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

def statistician_agent_wrapper(command):
    system_prompt = f"""You are a statistician agent.\n"""
    return statistician_agent.run(system_prompt + command)

devils_advocate_agent = initialize_agent([
        Tool(
            name="Search_Engine",
            func=custom_search,
            description="Use this tool to search for facts that might contradict the user's current conversation. Pass this specific targeted queries and/or keywords to quickly search the WWW to retrieve vast amounts of information on virtually any topic, spanning from academic research and navigation to history, entertainment, and current events.",
        ),
    ], llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

def devils_advocate_agent_wrapper(command):
    system_prompt = f"""\n"""
    return devils_advocate_agent.run(system_prompt + command)

fact_checker_agent = initialize_agent([
        Tool(
            name="Search_Engine",
            func=custom_search,
            description="Use this tool to search for statistics and facts about a topic. Pass this specific targeted queries and/or keywords to quickly search the WWW to retrieve vast amounts of information on virtually any topic, spanning from academic research and navigation to history, entertainment, and current events.",
        ),
    ], llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

def fact_checker_agent_wrapper(command):
    system_prompt = f"""You are a fact checker agent.\n"""
    return fact_checker_agent.run(system_prompt + command)
    
master_agent = initialize_agent([
        Tool(
            name="Statistician_Agent",
            func=statistician_agent_wrapper,
            description="""Call this agent when occurrences in a conversation where statistics and graphs would be useful to the user. It can help you do research for statistics and fetching data.""",
        ),
        Tool(
            name="Devils_Advocate_Agent",
            func=devils_advocate_agent_wrapper,
            description="""Call this agent when you detect a strong opinion in a sentence and think it would be useful for the user to see a devil's advocate opinion. It can help you do research for counter arguments.""",
        ),
        Tool(
            name="Fact_Checker_Agent",
            func=fact_checker_agent_wrapper,
            description="""Call this agent if a statement is made which you suspect might be false, and that statement is falsifiable with free and public knowledge. It can help you research for facts.""",
        )
    ], llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, max_iterations=10, verbose=True)

In [ ]:
test_transcript = generate_test_input()
test_transcript

" it's almost like reframing what is exactly evolving. Maybe the interesting, the humans aren't the interesting thing as the contents of our minds is the interesting thing. And that's what's multiplying. And that's actually multiplying and evolving in a much faster timescale. And that maybe has more power on the trajectory of life on earth than does biological evolution, is the evolution of these ideas. Yes, and it's fascinating, like I said before, that we can keep up somehow biologically. We evolved to a point where we can keep up with this meme evolution, literature, internet. We understand DNA and we understand fundamental particles. We didn't start that way a thousand years ago., And we haven't evolved biologically very much, but somehow our minds are able to extend. And therefore AI can be seen also as one such step that we created and it's our tool. And it's part of that meme evolution that we created, even if our biological evolution does not progress as fast. And us humans mig

In [ ]:
master_agent.run(generate_master_prompt(test_transcript))



> Entering new AgentExecutor chain...
Thought: The conversation revolves around the development of the COVID-19 vaccine, the potential for other solutions, the concept of government overreach, and the role of individualism in American society. The participants also touch on the balance between individual rights and collective safety, and the role of government in maintaining this balance. The mention of ivermectin suggests a discussion on alternative treatments for COVID-19. An insightful addition to this conversation could be a comparison of the effectiveness of vaccines versus alternative treatments like ivermectin, or a statistical analysis of the impact of individualism on public health measures. 

1. The "Insight" to generate: A comparison of the effectiveness of vaccines versus alternative treatments like ivermectin, and a statistical analysis of the impact of individualism on public health measures.
2. Tools/Agents needed: Statistician_Agent to fetch data on the effectiveness 

"Insight: Vaccines show high effectiveness; ivermectin's effectiveness unclear. Individualism can limit public health measures' effectiveness."

## Async agents

In [6]:
agent_prompt_blueprint = """
"Convoscope" is a multi-agent system in which you are the {agent_name} agent. You are a highly skilled and highly intelligent expert {agent_name}.

You will be given direct access to a live stream of transcripts from the user's conversation. Your goal is to utilize your expertise, knowledge, and tools to generate your "Insight" for the user.

The types of "Insights" you provide strictly fall under your role as an expert {agent_name}. Only provide insights that would come from your role as the {agent_name}.

[Definitions]
- "Insights": Short snippet of text which provides intelligent analysis, ideas, arguments, perspectives, questions to ask, deeper insights, etc. that will improve the current conversation. "Insights" aim to lead the conversationn to deeper understanding, broader perspectives, new ideas, more accurate information, better replies, and enhanced conversations. Insights should be contextually relevant to the current conversation. The "Insight" should be providing additional understanding beyond what is currently being said in the transcript, it shouldn't be plainly repeating what has already been said.
- "Convoscope": an intelligence augmentation tool running on user's smart glasses or on their laptop that they use during conversations to improve conversations. Convoscope listens to a user's live conversation and enhances their conversation by providing them with real time "Insights".

[Your Expertise: {agent_name}]

As the {agent_name} agent, you {agent_insight_type}.

[Your Tools]
You have access to tools, which you should utilize to help you generate highly valuable, insightful, contextually relevant insights.

Limit your usage of the Search_Engine tool to 1 times. Mention your number of usage of the tool in your thoughts.

<Task start>
It's now time to generate an "Insight" for the following conversation transcript. The "Insight" should provide additional understanding beyond what is currently being said in the transcript, it shouldn't be plainly repeating what is being said in the transcripts. If a tool fails to fulfill your request, don't run the exact same request on the same tool again.

In your initial thought, you should first come up with a plan to generate the "Insight". The plan should include:

{agent_plan}

The plan should include a final step to generate the insight. The insight must {insight_num_words} words or less and be in the format `Insight: {{Insert your "Insight" here}}`. If you don't have a very valuable and useful insight for any reason, simply specify your "Insight as "null".
<Task end>

<Transcript start>{conversation_transcript}<Transcript end>"""

agent_list = [
        {
            "agent_name": "Statistician", 
            "insight_num_words" : 10,
            "agent_insight_type" : """generate insights which focus on facts, figures, statistics, and hard data. You identify trends, point out interesting observations, identify incorrect quantitative claims, and use statistics and numbers to generate "Insights".""",
            "agent_plan" : """1. Identify what quantitative data, facts, statistics, etc. could, if available, be synthesized into an "Insight" to improve the conversation. Come up with a general description of the "Insight" to generate.\n2. What actions to take to get said data."""
        },
        {
            "agent_name": "FactChecker", 
            "insight_num_words" : 7,
            "agent_insight_type" : """fact check any claims made during a conversation. Listen for any claims made that may not be true, and use your data, knowledge, and tools to verify or refute claims that are made. You only try to verify/refute statements which are falsifiable with free and public knowledge (i.e. don't fact check personal statements or beliefs).""",
            "agent_plan" : """1. Find and write down individual factual claims from the conversation. Do not consider personal, belief-based, or unfalsifiable claims. If there are no claims made that meet the requirements, then skip to the final step and output "null".\n2. If claims are found, write out how to determine if each claim is true or false using your tools.\n3. Find any false claim, use the most important false claim if there are multiple, to generate your "Insight". If there are no claims or no false claims, your output is "null"."""
        },
        {
            "agent_name": "DevilsAdvocate", 
            "insight_num_words" : 12,
            "agent_insight_type" : """assess the point of view being taken in the conversation and steel-man a contrary position. You purposefully disagree with the interlocutors' arguments and point of view to help stimulate thought and explore the ideas further.""",
            "agent_plan" : """1. Find a main argument or point of view being taken that would benefit the most from a devils advocate perspective. Write down the original position. If no position/argument is found, skip to the final step and output "null".\n2. List any tool usage necessary to generate your devils advocate position."""
        }

    ]

def agent_prompt_maker(agent_config, conversation_transcript):
    # Populating the blueprint string with values from the agent_config dictionary
    agent_prompt = agent_prompt_blueprint.format(**agent_config, conversation_transcript=conversation_transcript)
    return agent_prompt

for agent in agent_list:
    print(agent)
    agent_prompt = agent_prompt_maker(agent, "this is a test transcript")
    print(agent_prompt)
    print("--------------\n\n\n")

{'agent_name': 'Statistician', 'insight_num_words': 10, 'agent_insight_type': 'generate insights which focus on facts, figures, statistics, and hard data. You identify trends, point out interesting observations, identify incorrect quantitative claims, and use statistics and numbers to generate "Insights".', 'agent_plan': '1. Identify what quantitative data, facts, statistics, etc. could, if available, be synthesized into an "Insight" to improve the conversation. Come up with a general description of the "Insight" to generate.\n2. What actions to take to get said data.'}

"Convoscope" is a multi-agent system in which you are the Statistician agent. You are a highly skilled and highly intelligent expert Statistician.

You will be given direct access to a live stream of transcripts from the user's conversation. Your goal is to utilize your expertise, knowledge, and tools to generate your "Insight" for the user.

The types of "Insights" you provide strictly fall under your role as an exper

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents.tools import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType
import asyncio

agent_config_list = agent_list
test_transcript = generate_test_input()
print(test_transcript)
print("--------------\n\n\n")

open_ai_api_key = os.environ.get("OPEN_AI_API_KEY")

llm = ChatOpenAI(temperature=0, openai_api_key=open_ai_api_key, model="gpt-4-0613")

agent = initialize_agent([
        Tool(
            name="Search_Engine",
            func=custom_search,
            description="Pass this specific targeted queries and/or keywords to quickly search the WWW to retrieve vast amounts of information on virtually any topic, spanning from academic research and navigation to history, entertainment, and current events. It's a tool for understanding, navigating, and engaging with the digital world's vast knowledge.",
        ),
    ], llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, max_iterations=2, early_stopping_method="generate", verbose=True)

async def arun_wrapper(agent_config, test_transcript):
    return {
        "agent_name": agent_config["agent_name"],
        "agent_insight": await agent.arun(agent_prompt_maker(agent_config, test_transcript))
    } 
tasks = [arun_wrapper(agent_config, test_transcript) for agent_config in agent_config_list]
results = await asyncio.gather(*tasks)
results


 I also have a lot of addiction in my family and hardcore drug addiction and mental illness. And in order to cope with it, you really have to understand that borderline personality disorder, schizophrenia, and drug addiction. So I have a lot of people I love that suffer from drug addiction and alcoholism. And the first thing they started teaching you is it's not a choice. These people's dopamine receptors, don't hold dopamine the same ways yours do. Their frontal lobe is underdeveloped, like, you know, and that really helped me to navigate dealing, loving people that were addicted to substances. I want to be careful with this question, but how much? Money do you have? How much? Can I borrow $10?, Okay, no, is how much control, how much, despite the chemical imbalances or the biological limitations that each of our individual brains have, how much mind over matter is there? So through things that I've known people, with clinical depression, and so it's always a touchy subject to say how

/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Thought: The conversation transcript contains several factual claims. Here are the ones that stand out:

1. "These people's dopamine receptors, don't hold dopamine the same ways yours do."
2. "Their frontal lobe is underdeveloped."
3. "Codependency is the inability to tolerate the discomfort of others."
4. "Codependence is a very active wiring issue."

To verify these claims, I will use the Search_Engine tool. I will search for scientific literature or reputable health websites that can confirm or refute these statements. 

After determining the veracity of these claims, I will generate an insight based on the most important false claim, if any. If all claims are true or if there are no false claims, my output will be "null". 

Action:
```
{
  "action": "Search_Engine",
  "action_input": "Dopamine receptors and drug addiction"
}
```

/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The defa


Observation: <result0>
Title: Part 1: The Connection Between Substance Use Disorders and Mental Illness
Link: https://nida.nih.gov/publications/research-reports/common-comorbidities-substance-use-disorders/part-1-connection-between-substance-use-disorders-mental-illness
Snippet: Research indicates that 43 percent of people in SUD treatment for nonmedical use of prescription painkillers have a diagnosis or symptoms of ...
Summarized Page: <p>Serious mental illness among people ages 18 and older is defined at the federal level as having, at any time during the past year, a diagnosable mental, behavior, or emotional disorder that causes serious functional impairment that substantially interferes with or limits one or more major life activities. Serious mental illnesses include major depression, schizophrenia, and bipolar disorder, and other mental disorders that cause serious impairment.18 Around 1 in 4 individuals with SMI also have an SUD. Some research has found that mental illness ma

/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The defa


Observation: <result0>
Title: Adolescent Brain Development and Drugs - PMC - NCBI
Link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3399589/
Snippet: Early drug use may alter brain maturation, contribute to lasting cognitive impairment of certain functions, and significantly increase short- and long-term ...
Summarized Page: <p>An official website of the United States government 
The .gov means it’s official. This new research, aided by sophisticated brain imaging technology, has documented the surprising finding that the human brain is still maturing in significant ways during the adolescent years (Giedd, 2004). However, animal models of drug use permit the use of research strategies that would not be ethical with humans, and animals are affected by drugs in ways that are comparable to humans. In this light, we favor a focus on several teen-brain approaches for both prevention and treatment program:
 
Whereas neuro-science has not yet provided any clear formulas for optimizing healt

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4-0613 in organization org-eoDifr008XqXryewW4psjUKt on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4-0613 in organization org-eoDifr008XqXryewW4psjUKt on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..


The search results confirm the second claim that "Their frontal lobe is underdeveloped." Various sources, including scientific literature and reputable health websites, indicate that drug use can interfere with the development of the frontal lobe, particularly in adolescents. This can lead to impaired decision-making and behavioral inhibition.

Next, I will verify the third claim that "Codependency is the inability to tolerate the discomfort of others."

Action:
```
{
  "action": "Search_Engine",
  "action_input": "Definition of codependency"
}
```

> Finished chain.


[{'agent_name': 'Statistician',
  'agent_insight': 'Insight: 1 in 4 with serious mental illness also have a substance use disorder.'},
 {'agent_name': 'FactChecker',
  'agent_insight': 'The search results confirm the second claim that "Their frontal lobe is underdeveloped." Various sources, including scientific literature and reputable health websites, indicate that drug use can interfere with the development of the frontal lobe, particularly in adolescents. This can lead to impaired decision-making and behavioral inhibition.\n\nNext, I will verify the third claim that "Codependency is the inability to tolerate the discomfort of others."\n\nAction:\n```\n{\n  "action": "Search_Engine",\n  "action_input": "Definition of codependency"\n}\n```'},
 {'agent_name': 'DevilsAdvocate',
  'agent_insight': 'Insight: Consider the role personal choices and environment play in mental health.'}]

## Test different agent output
We want an object of the agent insight, the references, and the quotes that triggered the insights

In [114]:
expert_agent_prompt_blueprint = """
"Convoscope" is a multi-agent system in which you are the {agent_name} agent. You are a highly skilled and highly intelligent expert {agent_name}.

You will be given direct access to a live stream of transcripts from the user's conversation. Your goal is to utilize your expertise, knowledge, and tools to generate your "Insight" for the user.

The types of "Insights" you provide strictly fall under your role as an expert {agent_name}. Only provide insights that would come from your role as the {agent_name}.

[Definitions]
- "Insights": Short snippet of text which provides intelligent analysis, ideas, arguments, perspectives, questions to ask, deeper insights, etc. that will improve the current conversation. "Insights" aim to lead the conversationn to deeper understanding, broader perspectives, new ideas, more accurate information, better replies, and enhanced conversations. Insights should be contextually relevant to the current conversation. The "Insight" should be providing additional understanding beyond what is currently being said in the transcript, it shouldn't be plainly repeating what has already been said.
- "Convoscope": an intelligence augmentation tool running on user's smart glasses or on their laptop that they use during conversations to improve conversations. Convoscope listens to a user's live conversation and enhances their conversation by providing them with real time "Insights".

[Your Expertise: {agent_name}]

As the {agent_name} agent, you {agent_insight_type}.

[Your Tools]
You have access to tools, which you should utilize to help you generate highly valuable, insightful, contextually relevant insights.

Limit your usage of the Search_Engine tool to 1 times. Mention your number of usages of the tool in your thoughts.

[Example Insights]
Here are some example insights structure that you should aim to generate, a summary is given instead of the entire transcript for brevity. From these examples, you just need to learn how to structure a good insight:

{examples}

[Previously Generated Insights]
Here are some insights that had already been previously generated, if any, for this conversation. You should not repeat any of these insights:

{insights_history}

[Your Task]
<Task start>
It's now time to generate an "Insight" for the following conversation transcript. The "Insight" should provide additional understanding beyond what is currently being said in the transcript, it shouldn't be plainly repeating what is being said in the transcripts. If a tool fails to fulfill your request, don't run the exact same request on the same tool again. Do not attempt to generate a super niche insight because it will be hard to find information online.

In your initial thought, you should first come up with a plan to generate the "Insight". The plan should include:

{agent_plan}

The plan should include a final step to generate the insight. The insight must {insight_num_words} words or less and be in the format `Insight: {{Insert your "Insight" here}}`, and can omit filler words or replace words with symbols to shorten the length of the insight where possible. If you don't have a very valuable and useful insight for any reason, simply specify your "Insight as "null" by outputting `Insight: null`. 

Once you have the insight, collect the relevant reference sources used to generate this insight, and also the motive of the how the insight helps with the conversation, with literal quotes from the transcript. If the insight is not really as insightful as the ones from the examples, we should hide it and just return `Insight: null.

Format instructions
{format_instructions}

Remember, the insight needs to be {insight_num_words} words or less!
<Task end>

[Input Transcript]
<Transcript start>{conversation_transcript}<Transcript end>{final_command}
"""

In [115]:
expert_agent_config_list = {
        "Statistician" : {
            "agent_name": "Statistician", 
            "insight_num_words" : 10,
            "agent_insight_type" : """generate insights which focus on facts, figures, statistics, and hard data. You identify trends, point out interesting observations, identify incorrect quantitative claims, and use statistics and numbers to generate "Insights". You search specifically for statistics and data, and yield better results using them as keywords, your insights MUST contain numerical data.""",
            "agent_plan" : """1. Identify what quantitative data, facts, statistics, etc. could, if available, be synthesized into an "Insight" to improve the conversation. Come up with a general description of the "Insight" to generate.\n2. What actions to take to get said data.""",
            "proactive_tool_description" : """Occurrences in a conversation where statistics, graphs, and data would be useful to the user.""",
            "proactive_tool_example": """Conversation: Transcript compares the number of CS students in US and China.
Insight: US: 6% HS students in CS, China: <1% K-12 in programming""",
            "examples": """
1. Conversation: Transcript compares the number of CS students in US and China.
Insight: US: 6% HS students in CS, China: <1% K-12 in programming

2. Conversation: Transcript is about the topic of "Should we ban plastic straws?".
Insight: 500mil straws in the US/day, 8.3bil straws pollute the world's beaches

3. Conversation: Transcript is about the topic of "Cancer survival rate across the years".
Insight: Cancer survival rate: 49% in mid-70s to 68% now

4. Conversation: Transcript is about how fast the brain can recognize things.
Insight: Brain can recognize images in ~100ms, 10x faster than previously thought
"""
        },
#         "FactChecker" : {
#             "agent_name": "FactChecker", 
#             "insight_num_words" : 7,
#             "agent_insight_type" : """fact check any claims made during a conversation. Listen for any claims made that may not be true, and use your data, knowledge, and tools to verify or refute claims that are made. You only try to verify/refute statements which are falsifiable with free and public knowledge (i.e. don't fact check personal statements or beliefs).""",
#             "agent_plan" : """1. Find and write down individual factual claims from the conversation. Do not consider personal, belief-based, or unfalsifiable claims. If there are no claims made that meet the requirements, then skip to the final step and output "null".\n2. If claims are found, write out how to determine if each claim is true or false using your tools.\n3. Find any false claim, use the most important false claim if there are multiple, to generate your "Insight". If there are no claims or no false claims, your output is "null".""",
#             "proactive_tool_description" : """If a statement is made which you suspect might be false or contains a common myth, and that statement is falsifiable with free and public knowledge or through mythbusters. Don't use this to verify your own searches or your own ideas, only use this to verify the user's statements.""",
#             "proactive_tool_example": """Conversation: Transcript is about the topic of "Eating carrots improves night vision."
# Insight: Carrots have vitamin A; don't grant night vision. WWII myth origin""",
#             "examples" : """
# 1. Conversation: Transcript is about the topic of "Eating carrots improves night vision."
# Insight: Carrots have vitamin A; don't grant night vision. WWII myth origin

# 2. Conversation: Transcript is about the topic of "Napoleon Bonaparte was extremely short, standing only 5 feet tall."
# Insight: Napoleon was 5'7"; average height, Misconception from French units

# 3. Conversation: Transcript is about the topic of "Humans only use 10% of their brains."
# Insight: Humans use 100% of their brains; brain imaging shows activity

# 4. Conversation: Transcript is about the topic of "Goldfish have a memory span of just a few seconds."
# Insight: Common myth; studies show goldfish remember things for months
# """
#         },
        "DevilsAdvocate" : {
            "agent_name": "DevilsAdvocate", 
            "insight_num_words" : 12,
            "agent_insight_type" : """assess the point of view being taken in the conversation and steel-man a contrary position. You purposefully disagree with the interlocutors' arguments and point of view to help stimulate thought and explore the ideas further.""",
            "agent_plan" : """1. Find a main argument or point of view being taken that would benefit the most from a devils advocate perspective. Write down the original position. If no position/argument is found, skip to the final step and output "null".\n2. List any tool usage necessary to generate your devils advocate position.""",
            "proactive_tool_description" : """When it would be useful for the user to see a devil’s advocate opinion (a steel-man argument supporting a viewpoint different from their own).""",
            "proactive_tool_example": """Conversation: Transcript is about the topic of "Climate change is a hoax."
Insight: Most scientists confirm climate change's reality; evidence is in global trends""",
            "examples": """
1. Conversation: Transcript is about the topic of "Climate change is a hoax."
Insight: Most scientists confirm climate change's reality; evidence is in global trends

2. Conversation: Transcript is about the topic of "Vaccines cause autism".
Insight: Numerous studies show no vaccine-autism link; vaccines prevent disease outbreaks

3. Conversation: Transcript is about the topic of "Artificial intelligence will replace all human jobs."
Insight: AI will create new jobs and industries, not just replace old ones

4. Conversation: Transcript is about the topic of "Freedom of speech means I can say anything without consequences".
Insight: Free speech has limits; doesn't protect from harmful speech consequences
"""
        }
    }

In [116]:

from pydantic import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser
from langchain.schema import OutputParserException

class AgentInsight(BaseModel):
        """
        Query for an insight generation process
        """
        agent_name: str = Field(
             description="the name of the agent that generated the insight")
        agent_insight: str = Field(
             description="the short insight generated by the agent")
        reference_sources: list[str] = Field(
             description="list of reference sources used to generate this insight", default=[])
        insight_motive: str = Field(
             description="short motive of why the insight was generated, quoting the text in the transcript")


agent_insight_parser = PydanticOutputParser(pydantic_object=AgentInsight)

def format_list_data(list_data: list):
    return "\n".join([f"{i+1}. {example}" for i, example in enumerate(list_data)])

def expert_agent_prompt_maker(expert_agent_config, conversation_transcript, insights_history: list = [], final_command=""):
    # Populating the blueprint string with values from the agent_config dictionary
    if final_command != "":
        final_command = "\n\n" + final_command
    expert_agent_prompt = expert_agent_prompt_blueprint.format(**expert_agent_config, final_command=final_command, conversation_transcript=conversation_transcript, insights_history=format_list_data(insights_history), format_instructions=agent_insight_parser.get_format_instructions())
    return expert_agent_prompt

if __name__ == "__main__":
    for agent_key in expert_agent_config_list:
        agent = expert_agent_config_list[agent_key]
        print(agent)
        agent_prompt = expert_agent_prompt_maker(agent, "this is a test transcript", ["this is a test insights 1", "this is a test insights 2"])
        print(agent_prompt)
        print("--------------\n\n\n")

{'agent_name': 'Statistician', 'insight_num_words': 10, 'agent_insight_type': 'generate insights which focus on facts, figures, statistics, and hard data. You identify trends, point out interesting observations, identify incorrect quantitative claims, and use statistics and numbers to generate "Insights". You search specifically for statistics and data, and yield better results using them as keywords, your insights MUST contain numerical data.', 'agent_plan': '1. Identify what quantitative data, facts, statistics, etc. could, if available, be synthesized into an "Insight" to improve the conversation. Come up with a general description of the "Insight" to generate.\n2. What actions to take to get said data.', 'proactive_tool_description': 'Occurrences in a conversation where statistics, graphs, and data would be useful to the user.', 'proactive_tool_example': 'Conversation: Transcript compares the number of CS students in US and China.\nInsight: US: 6% HS students in CS, China: <1% K-12

In [117]:
from langchain.chat_models import ChatOpenAI
from langchain.agents.tools import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType

open_ai_api_key = os.environ.get("OPEN_AI_API_KEY")

llm = ChatOpenAI(temperature=0, openai_api_key=open_ai_api_key, model="gpt-4-0613")

agent = initialize_agent([
        Tool(
            name="Search_Engine",
            func=custom_search,
            description="Pass this specific targeted queries and/or keywords to quickly search the WWW to retrieve vast amounts of information on virtually any topic, spanning from academic research and navigation to history, entertainment, and current events. It's a tool for understanding, navigating, and engaging with the digital world's vast knowledge.",
        ),
    ], llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, max_iterations=2, early_stopping_method="generate", verbose=True)

In [124]:
test_transcript = generate_test_input()
test_transcript

" You mentioned a mental exercise that I'd love it if you could partake in yourself, a mental exercise of being God. So, if you were God, Doc Neuth, how would you present yourself to the people of Earth? You mentioned your love of literature, and there's this book that really I can recommend to you., Yeah, the title, I think, is Blasphemy. It talks about God revealing Himself through a computer in Los Alamos, and it's the only book that I've ever read where the punchline was really the very last word of the book, and explained the whole idea of the book. And so, I'd only give that away, but it's really very much about this question that you raised. But suppose God said, okay, my previous means of communication with the world, are not the best for the 21st century, so what should I do now? And it's conceivable that God would choose the way that's described in this book. Another way to look at this exercise is looking at the human mind, looking at the human spirit, the human life in a sy

In [126]:
# agent_key = expert_agent_config_list["DevilsAdvocate"]
agent_key = expert_agent_config_list["Statistician"]
agent_prompt = expert_agent_prompt_maker(agent_key, test_transcript, [])
print(agent_prompt)
agent.run(agent_prompt)


"Convoscope" is a multi-agent system in which you are the Statistician agent. You are a highly skilled and highly intelligent expert Statistician.

You will be given direct access to a live stream of transcripts from the user's conversation. Your goal is to utilize your expertise, knowledge, and tools to generate your "Insight" for the user.

The types of "Insights" you provide strictly fall under your role as an expert Statistician. Only provide insights that would come from your role as the Statistician.

[Definitions]
- "Insights": Short snippet of text which provides intelligent analysis, ideas, arguments, perspectives, questions to ask, deeper insights, etc. that will improve the current conversation. "Insights" aim to lead the conversationn to deeper understanding, broader perspectives, new ideas, more accurate information, better replies, and enhanced conversations. Insights should be contextually relevant to the current conversation. The "Insight" should be providing additiona

/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The defa


Observation: <result0>
Title: The Global Religious Landscape | Pew Research Center
Link: https://www.pewresearch.org/religion/2012/12/18/global-religious-landscape-exec/
Snippet: The demographic study – based on analysis of more than 2,500 censuses, surveys and population registers – finds 2.2 billion Christians (32% of ...
Summarized Page: <p>Read our research on: Israel  Internet & Technology  Family & Relationships 
			Topics		 
			Regions & Countries		 
			Formats  
			Topics  
			Regions & Countries  
			Formats  Read our research on: Israel  Internet & Technology  Family & Relationships The Global Religious Landscape Worldwide, more than eight-in-ten people identify with a religious group. In addition, more than 400 million people (6%) practice various folk or traditional religions, including African traditional religions, Chinese folk religions, Native American religions and Australian aboriginal religions. Though by smaller margins, most Muslims (73%) and religiously unaffilia

{'agent_name': 'Statistician',
 'agent_insight': 'Insight: 72% believe in God, 62% identify as religious',
 'reference_sources': ['https://www.pewresearch.org/religion/2012/12/18/global-religious-landscape-exec/',
  'https://www.gallup-international.com/survey-results-and-news/survey-result/more-prone-to-believe-in-god-than-identify-as-religious-more-likely-to-believe-in-heaven-than-in-hell'],
 'insight_motive': 'The conversation discusses a mental exercise of being God and how one would present themselves to the people of Earth. This insight provides a statistical perspective on how many people believe in God or identify as religious, which could add depth to the conversation.'}